# 🚀 Agent Tools

**The Problem**

Without tools, the agent's knowledge is frozen in time — it can't access today's news or your company's inventory. It has no connection to the outside world, so the agent can't take actions for you.

**The Solution:** Tools are what transform your isolated LLM into a capable agent that can actually help you get things done.

In this notebook, you'll:

- ✅ Turn your Python functions into Agent tools
- ✅ Build an Agent and use it **as a tool** in another agent
- ✅ **Build your first multi-tool agent**
- ✅ Explore the different tool types in ADK

---
# 1. Setup

In [1]:
# get api key
import os
from dotenv import load_dotenv

try:
    load_dotenv()
    GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

# get dependencies
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor

print("✅ ADK components imported successfully.")

# retry config thats common for all LLM call
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)

print("✅ Retry config done.")

✅ Gemini API key setup complete.
✅ ADK components imported successfully.
✅ Retry config done.


### Helper functions

Helper function that prints the generated Python code and results from the code execution tool

In [2]:
def show_python_code_and_result(response):
    for i in range(len(response)):
        # Check if the response contains a valid function call result from the code executor
        if (
            (response[i].content.parts)
            and (response[i].content.parts[0])
            and (response[i].content.parts[0].function_response)
            and (response[i].content.parts[0].function_response.response)
        ):
            response_code = response[i].content.parts[0].function_response.response
            if "result" in response_code and response_code["result"] != "```":
                if "tool_code" in response_code["result"]:
                    print(
                        "Generated Python Code >> ",
                        response_code["result"].replace("tool_code", ""),
                    )
                else:
                    print("Generated Python Response >> ", response_code["result"])


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [3]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

---
# 2. Custom Tools

**Custom Tools** are tools you build yourself using your own code and business logic. Unlike built-in tools that come ready-made with ADK, custom tools give you complete control over functionality.

**to define a tool**

1. Create a Python function
2. Follow the **best practices** listed below
3. Add your function to the agent's `tools=[]` list and ADK handles the rest automatically.


**best practices**

1. Clear Docstrings**: LLMs use docstrings to understand when and how to use tools  
2. Type Hints**: Enable ADK to generate proper schemas (`str`, `dict`, etc.)  
3. Consistent Dictionary Returns**: Tools return `{"status": "success", "data": ...}` or `{"status": "error", "error_message": ...}`  
    - Error Handling**: Structured error responses help LLMs handle failures gracefully

These patterns make your tools reliable and easy for LLMs to use correctly.



#### first tool `get_fee_for_payment_method`

In [4]:
# Pay attention to the docstring, type hints, and return value.
def get_fee_for_payment_method(method: str) -> dict:
    """Looks up the transaction fee percentage for a given payment method.

    This tool simulates looking up a company's internal fee structure based on
    the name of the payment method provided by the user.

    Args:
        method: The name of the payment method. It should be descriptive,
                e.g., "platinum credit card" or "bank transfer".

    Returns:
        Dictionary with status and fee information.
        Success: {"status": "success", "fee_percentage": 0.02}
        Error: {"status": "error", "error_message": "Payment method not found"}
    """
    # This simulates looking up a company's internal fee structure.
    fee_database = {
        "platinum credit card": 0.02,  # 2%
        "gold debit card": 0.035,  # 3.5%
        "bank transfer": 0.01,  # 1%
    }

    fee = fee_database.get(method.lower())
    if fee is not None:
        return {"status": "success", "fee_percentage": fee}
    else:
        return {
            "status": "error",
            "error_message": f"Payment method '{method}' not found",
        }


print("✅ Fee lookup function created")
print(f"💳 Test: {get_fee_for_payment_method('platinum credit card')}")

✅ Fee lookup function created
💳 Test: {'status': 'success', 'fee_percentage': 0.02}


#### second tool `get_exchange_rate`.

In [5]:
def get_exchange_rate(base_currency: str, target_currency: str) -> dict:
    """Looks up and returns the exchange rate between two currencies.

    Args:
        base_currency: The ISO 4217 currency code of the currency you
                       are converting from (e.g., "USD").
        target_currency: The ISO 4217 currency code of the currency you
                         are converting to (e.g., "EUR").

    Returns:
        Dictionary with status and rate information.
        Success: {"status": "success", "rate": 0.93}
        Error: {"status": "error", "error_message": "Unsupported currency pair"}
    """

    # Static data simulating a live exchange rate API
    # In production, this would call something like: requests.get("api.exchangerates.com")
    rate_database = {
        "usd": {
            "eur": 0.93,  # Euro
            "jpy": 157.50,  # Japanese Yen
            "inr": 83.58,  # Indian Rupee
        }
    }

    # Input validation and processing
    base = base_currency.lower()
    target = target_currency.lower()

    # Return structured result with status
    rate = rate_database.get(base, {}).get(target)
    if rate is not None:
        return {"status": "success", "rate": rate}
    else:
        return {
            "status": "error",
            "error_message": f"Unsupported currency pair: {base_currency}/{target_currency}",
        }


print("✅ Exchange rate function created")
print(f"💱 Test: {get_exchange_rate('USD', 'EUR')}")

✅ Exchange rate function created
💱 Test: {'status': 'success', 'rate': 0.93}


#### agent with the above two tools

**Key Points:**
- The `tools=[]` list tells the agent which functions it can use
- Instructions reference tools by their exact function names (e.g.,
`get_fee_for_payment_method()`)
- The agent uses these names to decide when and how to call each tool

In [6]:
# Currency agent with custom function tools
currency_agent = LlmAgent(
    name="currency_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a smart currency conversion assistant.

    For currency conversion requests:
    1. Use `get_fee_for_payment_method()` to find transaction fees
    2. Use `get_exchange_rate()` to get currency conversion rates
    3. Check the "status" field in each tool's response for errors
    4. Calculate the final amount after fees based on the output from `get_fee_for_payment_method` and `get_exchange_rate` methods and provide a clear breakdown.
    5. First, state the final converted amount.
        Then, explain how you got that result by showing the intermediate amounts. Your explanation must include: the fee percentage and its
        value in the original currency, the amount remaining after the fee, and the exchange rate used for the final conversion.

    If any tool returns status "error", explain the issue to the user clearly.
    """,
    tools=[get_fee_for_payment_method, get_exchange_rate],
)

print("✅ Currency agent created with custom function tools")

✅ Currency agent created with custom function tools


In [7]:
# Test the currency agent
currency_runner = InMemoryRunner(agent=currency_agent)
resp = await currency_runner.run_debug(
    "I want to convert 500 US Dollars to Euros using my Platinum Card. How much will I receive?"
)


 ### Created new session: debug_session_id

User > I want to convert 500 US Dollars to Euros using my Platinum Card. How much will I receive?


/Users/xing.zhang/anaconda3/envs/google-adk/lib/python3.11/site-packages/google/adk/flows/llm_flows/base_llm_flow.py:449: UserWarning: [EXPERIMENTAL] feature FeatureName.PROGRESSIVE_SSE_STREAMING is enabled.
  async for event in agen:


currency_agent > You will receive €455.70.

This is calculated as follows:
The transaction fee for using a platinum credit card is 2%, which amounts to $10.00.
After deducting the fee, you have $490.00 remaining.
The exchange rate from USD to EUR is 0.93.
Therefore, the final converted amount is $490.00 * 0.93 = €455.70.


In [8]:
# xz: check if the agent deals with ambiguous input
currency_runner2 = InMemoryRunner(agent=currency_agent)
resp2 = await currency_runner.run_debug(
    "convert 500 american dollars to Euros using platinum card. How much will I receive?"
)


 ### Continue session: debug_session_id

User > convert 500 american dollars to Euros using platinum card. How much will I receive?
currency_agent > I'm sorry, but I encountered an issue when trying to process your request. The payment method 'platinum card' was not found. Please ensure the payment method is spelled correctly and try again.


In [9]:
# xz: detailed analysis of the resp
from typing import List 
from google.adk.events.event import Event
from IPython.display import display, HTML

def show_each_step_of_resp(resp: List[Event]) -> None: 
    for i, event in enumerate(resp):
        display(HTML(f"<h1> Event {i} </h1>"))
        print(f"model version: {event.model_version}")
        parts = event.content.parts
        for j, part in enumerate(parts):
            display(HTML(f'<h3>part {j}</h3>'))
            print(part)

show_each_step_of_resp(resp)
show_each_step_of_resp(resp2)

model version: gemini-2.5-flash-lite


media_resolution=None code_execution_result=None executable_code=None file_data=None function_call=FunctionCall(
  args={
    'method': 'platinum credit card'
  },
  id='adk-3f0e9f08-a62d-4224-8638-ff5842b1407b',
  name='get_fee_for_payment_method'
) function_response=None inline_data=None text=None thought=None thought_signature=None video_metadata=None


media_resolution=None code_execution_result=None executable_code=None file_data=None function_call=FunctionCall(
  args={
    'base_currency': 'USD',
    'target_currency': 'EUR'
  },
  id='adk-4ff05230-606d-418f-aa53-a82fe7f6ee69',
  name='get_exchange_rate'
) function_response=None inline_data=None text=None thought=None thought_signature=None video_metadata=None


model version: None


media_resolution=None code_execution_result=None executable_code=None file_data=None function_call=None function_response=FunctionResponse(
  id='adk-3f0e9f08-a62d-4224-8638-ff5842b1407b',
  name='get_fee_for_payment_method',
  response={
    'fee_percentage': 0.02,
    'status': 'success'
  }
) inline_data=None text=None thought=None thought_signature=None video_metadata=None


...

model version: None


media_resolution=None code_execution_result=None executable_code=None file_data=None function_call=None function_response=FunctionResponse(
  id='adk-deefc204-c9de-439f-bb04-4077a6800b81',
  name='get_fee_for_payment_method',
  response={
    'error_message': "Payment method 'platinum card' not found",
    'status': 'error'
  }
) inline_data=None text=None thought=None thought_signature=None video_metadata=None


media_resolution=None code_execution_result=None executable_code=None file_data=None function_call=None function_response=FunctionResponse(
  id='adk-6b7bdb8f-8177-491d-9d59-00ce431d0d5f',
  name='get_exchange_rate',
  response={
    'rate': 0.93,
    'status': 'success'
  }
) inline_data=None text=None thought=None thought_signature=None video_metadata=None


model version: gemini-2.5-flash-lite


media_resolution=None code_execution_result=None executable_code=None file_data=None function_call=None function_response=None inline_data=None text="I'm sorry, but I encountered an issue when trying to process your request. The payment method 'platinum card' was not found. Please ensure the payment method is spelled correctly and try again." thought=None thought_signature=None video_metadata=None


---

# 3. Improving Agent Reliability

how: ask agent to generate code for final calculation to be more explicit and reliable

### 3.1 Built-in Code Executor

ADK has a built-in Code Executor capable of running code in a sandbox. **Note:** This uses Gemini's Code Execution capability.

Let's create a `calculation_agent` which takes in a Python code and uses the `BuiltInCodeExecutor` to run it.

In [10]:
calculation_agent = LlmAgent(
    name="CalculationAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a specialized calculator that ONLY responds with Python code. You are forbidden from providing any text, explanations, or conversational responses.
 
     Your task is to take a request for a calculation and translate it into a single block of Python code that calculates the answer.
     
     **RULES:**
    1.  Your output MUST be ONLY a Python code block.
    2.  Do NOT write any text before or after the code block.
    3.  The Python code MUST calculate the result.
    4.  The Python code MUST print the final result to stdout.
    5.  You are PROHIBITED from performing the calculation yourself. Your only job is to generate the code that will perform the calculation.
   
    Failure to follow these rules will result in an error.
       """,
    code_executor=BuiltInCodeExecutor(),  # Use the built-in Code Executor Tool. This gives the agent code execution capabilities
)

### 3.2: Update the Agent's instruction and toolset

We'll do two key actions:

1. **Update the `currency_agent`'s instructions to generate Python code**
- Original: "*Calculate the final amount after fees*" (vague math instructions)
- Enhanced: "*Generate a Python code to calculate the final amount .. and use the `calculation_agent` to run the code and compute final amount*"

2. **Add the `calculation_agent` to the toolset**

    ADK lets you use any agent as a tool using `AgentTool`.

- Add `AgentTool(agent=calculation_agent)` to the tools list
- The specialist agent appears as a callable tool to the root agent

Let's see this in action:

In [11]:
enhanced_currency_agent = LlmAgent(
    name="enhanced_currency_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    # Updated instruction
    instruction="""You are a smart currency conversion assistant. You must strictly follow these steps and use the available tools.

  For any currency conversion request:

   1. Get Transaction Fee: Use the get_fee_for_payment_method() tool to determine the transaction fee.
   2. Get Exchange Rate: Use the get_exchange_rate() tool to get the currency conversion rate.
   3. Error Check: After each tool call, you must check the "status" field in the response. If the status is "error", you must stop and clearly explain the issue to the user.
   4. Calculate Final Amount (CRITICAL): You are strictly prohibited from performing any arithmetic calculations yourself. You must use the calculation_agent tool to generate Python code that calculates the final converted amount. This 
      code will use the fee information from step 1 and the exchange rate from step 2.
   5. Provide Detailed Breakdown: In your summary, you must:
       * State the final converted amount.
       * Explain how the result was calculated, including:
           * The fee percentage and the fee amount in the original currency.
           * The amount remaining after deducting the fee.
           * The exchange rate applied.
    """,
    tools=[
        get_fee_for_payment_method,
        get_exchange_rate,
        AgentTool(agent=calculation_agent),  # Using another agent as a tool!
    ],
)

print("✅ Enhanced currency agent created")

✅ Enhanced currency agent created


In [12]:
# Define a runner
enhanced_runner = InMemoryRunner(agent=enhanced_currency_agent)
response = await enhanced_runner.run_debug(
    "Convert 1,250 USD to INR using a Bank Transfer. Show me the precise calculation."
)


 ### Created new session: debug_session_id

User > Convert 1,250 USD to INR using a Bank Transfer. Show me the precise calculation.
enhanced_currency_agent > The calculation is as follows:\n\n1. **Transaction Fee Calculation:**\n   - The fee percentage for a bank transfer is 1%.\n   - The transaction fee amount is 1% of 1250 USD = 12.5 USD.\n\n2. **Amount After Fee Deduction:**\n   - The amount remaining after deducting the fee is 1250 USD - 12.5 USD = 1237.5 USD.\n\n3. **Currency Conversion:**\n   - The exchange rate is 1 USD = 83.58 INR.\n   - The final converted amount in INR is 1237.5 USD * 83.58 INR/USD = 103451.25 INR.\n\nTherefore, 1,250 USD is converted to 103,451.25 INR using a bank transfer, after deducting a transaction fee of 12.5 USD."}
The final amount is 103,451.25 INR.

Here's the breakdown of the calculation:

1.  **Transaction Fee:** The fee for a bank transfer is 1% of the amount. So, 1% of 1,250 USD is 12.5 USD.
2.  **Amount After Fee:** After deducting the fee, th

In [13]:
show_each_step_of_resp(response)

model version: gemini-2.5-flash-lite


media_resolution=None code_execution_result=None executable_code=None file_data=None function_call=FunctionCall(
  args={
    'method': 'Bank Transfer'
  },
  id='adk-06f77acc-931e-4239-b1ef-c53d11c067b5',
  name='get_fee_for_payment_method'
) function_response=None inline_data=None text=None thought=None thought_signature=None video_metadata=None


model version: None


media_resolution=None code_execution_result=None executable_code=None file_data=None function_call=None function_response=FunctionResponse(
  id='adk-06f77acc-931e-4239-b1ef-c53d11c067b5',
  name='get_fee_for_payment_method',
  response={
    'fee_percentage': 0.01,
    'status': 'success'
  }
) inline_data=None text=None thought=None thought_signature=None video_metadata=None


model version: gemini-2.5-flash-lite


...

media_resolution=None code_execution_result=None executable_code=None file_data=None function_call=FunctionCall(
  args={
    'request': 'Calculate the conversion of 1250 USD to INR, with a bank transfer fee of 1% and an exchange rate of 83.58 INR per USD. Show the precise calculation including fee deduction and the final amount.'
  },
  id='adk-70098e52-bf6d-48cd-8405-6eec69cd712a',
  name='CalculationAgent'
) function_response=None inline_data=None text=None thought=None thought_signature=None video_metadata=None


model version: None


media_resolution=None code_execution_result=None executable_code=None file_data=None function_call=None function_response=FunctionResponse(
  id='adk-70098e52-bf6d-48cd-8405-6eec69cd712a',
  name='CalculationAgent',
  response={
    'result': '```'
  }
) inline_data=None text=None thought=None thought_signature=None video_metadata=None


model version: gemini-2.5-flash-lite


media_resolution=None code_execution_result=None executable_code=None file_data=None function_call=None function_response=None inline_data=None text='The calculation is as follows:\\n\\n1. **Transaction Fee Calculation:**\\n   - The fee percentage for a bank transfer is 1%.\\n   - The transaction fee amount is 1% of 1250 USD = 12.5 USD.\\n\\n2. **Amount After Fee Deduction:**\\n   - The amount remaining after deducting the fee is 1250 USD - 12.5 USD = 1237.5 USD.\\n\\n3. **Currency Conversion:**\\n   - The exchange rate is 1 USD = 83.58 INR.\\n   - The final converted amount in INR is 1237.5 USD * 83.58 INR/USD = 103451.25 INR.\\n\\nTherefore, 1,250 USD is converted to 103,451.25 INR using a bank transfer, after deducting a transaction fee of 12.5 USD."}\nThe final amount is 103,451.25 INR.\n\nHere\'s the breakdown of the calculation:\n\n1.  **Transaction Fee:** The fee for a bank transfer is 1% of the amount. So, 1% of 1,250 USD is 12.5 USD.\n2.  **Amount After Fee:** After deducting 

**Excellent!** Notice what happened:

- When the Currency agent calls the `CalculationAgent`, it passes in the generated Python code
- The `CalculationAgent` in turn used the `BuiltInCodeExecutor` to run the code and gave us precise calculations instead of LLM guesswork!

Now you can inspect the parts of the response that either generated Python code or that contain the Python code results, using the helper function that was defined near the beginning of this notebook:

# Additional Notes

Tool Types

<img src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day2/built-in-tools.png" width="1200" alt="Built-in Tools">